# Notebook: Filter Sentences from Reviews to create Dataset for Annotation

## Packages

In [1]:
from bs4 import BeautifulSoup
from langdetect import detect
import pandas as pd
import json
import nltk
import uuid
from nltk.tokenize import sent_tokenize
import re

## Settings

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nils_hellwig/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Constants

In [3]:
REVIEWS_PATH = "reviews_dataset/reviews.csv"
RANDOM_STATE = 43

## Code

### Load Dataset

In [4]:
df_reviews = pd.read_csv(REVIEWS_PATH)

In [5]:
df_reviews

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization
0,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland","Die LOC schmeckte leicht säuerlich, der kleine...",2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...
1,1,907168709,2005104,0,Wir sind hier nicht in Bayern!,2023-08-01,173stefanh,"München, Deutschland","Wir sind hier nicht in LOC, sind die freundlic...",1.0,berlin,Augustiner am Gendarmenmarkt,de,"Wir sind hier nicht in Bayern, sind die freund..."
2,2,902540630,2005104,0,Heute leider zum abgewöhnen,2023-07-12,Patrick L,"Rüdersdorf, Deutschland",Ich freute mich sehr auf das Wienerschnitzel u...,3.0,berlin,Augustiner am Gendarmenmarkt,de,Ich freute mich sehr auf das Wienerschnitzel u...
3,3,895783388,2005104,0,Teils-Teils,2023-06-18,J9324NNpeterr,"Stellenbosch, Südafrika",Meine Erfahrungen in diesem Jahr waren gemisch...,4.0,berlin,Augustiner am Gendarmenmarkt,de,Meine Erfahrungen in diesem Jahr waren gemisch...
4,4,890917149,2005104,0,schlecht verwaltetes Chaos,2023-05-18,Harald F,"Berlin, Deutschland",ich hätte gerne etwas zum Essen gesagt. Aber e...,1.0,berlin,Augustiner am Gendarmenmarkt,de,ich hätte gerne etwas zum Essen gesagt. Aber e...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3596,3596,889964622,5207419,0,Nicht besonderes,2023-05-11,Mobile28959210843,"Mönchengladbach, Deutschland",Das Essen ist okay mehr aber auch nicht. Ambie...,3.0,mönchengladbach,Brauerei zum Stefanus,de,Das Essen ist okay mehr aber auch nicht. Ambie...
3597,3597,870345760,5207419,0,"Ungebührender Empfang, sehr gutes Essen",2022-12-01,Korich,"Görlitz, Deutschland",Zunächst einmal wurden wir von einem Mann mit ...,4.0,mönchengladbach,Brauerei zum Stefanus,de,Zunächst einmal wurden wir von einem Mann mit ...
3598,3598,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Wir sind nicht zum ersten Mal aber auch defini...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...
3599,3599,863841135,5207419,0,Gutes Essen mit Biergartenambiente,2022-10-09,788heinzn,"Köln, Deutschland","Gutes Essen, Portionen sehr groß (für uns zu g...",4.0,mönchengladbach,Brauerei zum Stefanus,de,"Gutes Essen, Portionen sehr groß (für uns zu g..."


### Filter Sentences

In [6]:
df_reviews_sentences = pd.DataFrame(columns=list(df_reviews.columns) + ['sentence_idx', "id"])

for idx, row in df_reviews.iterrows():
    sentences = sent_tokenize(row['text'], language="german")
    sentence_index = 0
    for sentence in sentences:
        if len(sentence) > 1:
            new_row = row.copy() 
            new_row['text'] = sentence 
            new_row['id'] = str(uuid.uuid4())
            new_row['sentence_idx'] = sentence_index
            df_reviews_sentences.loc[len(df_reviews_sentences)] = new_row
            sentence_index += 1

In [7]:
df_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id
0,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland","Die LOC schmeckte leicht säuerlich, der kleine...",2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,0,3868e044-e85a-4746-8608-5b86677ac8d2
1,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland",Der Schweinebraten war lauwarm und der Kaisers...,2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,1,ceb56407-821b-4832-a203-5fd8e342d6d4
2,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland",Ich bin so enttäuschst.,2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,2,e5ef1420-2485-426f-8b78-787473c241f8
3,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland",Gehe ich nicht mehr hin.,2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,3,ab29a0e6-46d7-4d02-b271-aba796b84fa0
4,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland","Schlecht kann ich auch woanders, besonders bei...",2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,4,673770f9-a45f-4a55-b46d-f74a5add8520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15344,3600,861397747,5207419,0,Das Schnitzelparadies!,2022-09-22,vereny2020,NaN,Das Essen schmeckt aber generell grandios.,5.0,mönchengladbach,Brauerei zum Stefanus,de,Ich bin im Schnitzelparadies <3 Die Schnitzel ...,1,abb2d790-754c-4c9f-94ea-2bacb1c700f3
15345,3600,861397747,5207419,0,Das Schnitzelparadies!,2022-09-22,vereny2020,NaN,Der Beilagensalat ist auch der Kracher: frisch...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Ich bin im Schnitzelparadies <3 Die Schnitzel ...,2,f385125a-30a0-432b-a388-5988c490e8b1
15346,3600,861397747,5207419,0,Das Schnitzelparadies!,2022-09-22,vereny2020,NaN,"Im Stefanus gibt es auch selbstgebrautes, sehr...",5.0,mönchengladbach,Brauerei zum Stefanus,de,Ich bin im Schnitzelparadies <3 Die Schnitzel ...,3,51d0a800-0d74-4bd7-996a-f8ab78c7e14d
15347,3600,861397747,5207419,0,Das Schnitzelparadies!,2022-09-22,vereny2020,NaN,"Der Service ist sehr nett, zuvorkommend und he...",5.0,mönchengladbach,Brauerei zum Stefanus,de,Ich bin im Schnitzelparadies <3 Die Schnitzel ...,4,2c9bc1a2-0a1f-4bb7-9e7f-f152328f1254


### Filter Non-German Sentences

Even if the language code = "de", we have observed that some sentences are in other languagesthan german. These are excluded as well with the help of Google's language-detection.

In [8]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'Unknown'

df_reviews_sentences['detected_language'] = df_reviews_sentences['text'].apply(detect_language)
df_reviews_sentences[df_reviews_sentences['detected_language'] != 'de']

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
17,4,890917149,2005104,0,schlecht verwaltetes Chaos,2023-05-18,Harald F,"Berlin, Deutschland",Es war viel los.,1.0,berlin,Augustiner am Gendarmenmarkt,de,ich hätte gerne etwas zum Essen gesagt. Aber e...,3,2cebc712-5d46-48c4-abc4-277807485310,af
66,13,866452621,2005104,1,Mind this touri-gap,2022-10-29,452stephand,"Nürnberg, Deutschland",Fazit: mind this touri-gap,1.0,berlin,Augustiner am Gendarmenmarkt,de,Wunderschöne Location mit mangelhafter Leitung...,1,ef1bfbed-a7ad-47bd-8bc8-209050ed13c7,en
93,17,856900319,2005104,1,Deftig bayrisch,2022-08-28,tatrab,"Kaiserslautern, Deutschland",Kann man mal hin,3.0,berlin,Augustiner am Gendarmenmarkt,de,Vorzubereitete Speisen in bayrischer Manier zu...,2,f7475a9c-cb4f-48d7-9fc7-9fb6c0612329,tl
113,22,847064431,2005104,2,Enttäuschend,2022-07-09,HerSchnu,"Erfurt, Deutschland",Fing schon gut an….,2.0,berlin,Augustiner am Gendarmenmarkt,de,In jedem bayerischem Gasthof isst man besser… ...,2,0935e297-1475-49be-866f-70a30db018d6,en
131,25,883653345,947082,0,Tolle Burger!,2023-03-26,dittelbrunn,"Lübeck, Deutschland",Ketchup.,5.0,berlin,Burgermeister,de,Wir waren in der Filiale am S-Bahnhof Eberswal...,2,4dc97dbc-595c-4950-aab1-e4c0aa126b46,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15231,3570,862649725,13144674,6,Einfach eine Freude und ein Genuss,2022-09-30,Bernd W,NaN,Klasse.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,"Okay, ich koche gern selbst. Wenn ich aber mal...",4,c0568b45-62d2-4c52-8949-c6ea3e25dde3,et
15241,3570,862649725,13144674,6,Einfach eine Freude und ein Genuss,2022-09-30,Bernd W,NaN,Place to be.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,"Okay, ich koche gern selbst. Wenn ich aber mal...",14,8c2be4a6-c652-4ac7-aef2-0abffa31fbbf,en
15249,3572,861638375,13144674,6,"Handwerklich einwandfrei, Hohe Qualität und ei...",2022-09-24,Tilmann S,NaN,Top.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Eher zufällig kehrten wir als Ortsunkundige im...,4,05080ec4-42bf-401a-9680-7c7328b4671c,nl
15280,3583,847136487,13144674,7,Super Essen Super Service,2022-07-10,U4608RGalexs,"Schwalbach, Deutschland",Alles was wir essen war super.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Hervorragend wie immer. Immer wenn wir aus Fra...,2,ffa11b40-abf0-4f7b-a90a-78ed8e22f773,en


In [9]:
df_reviews_sentences = df_reviews_sentences[df_reviews_sentences['detected_language'] == 'de']

### Balancing

In [10]:
df_balanced_reviews_sentences = df_reviews_sentences.groupby(['rating'], group_keys=False).apply(lambda x: x.sample(700, replace=False, random_state=RANDOM_STATE))
df_balanced_reviews_sentences.reset_index(drop=True, inplace=True)

### Randomisierung

In [11]:
df_balanced_reviews_sentences = df_balanced_reviews_sentences.sample(frac=1, random_state=RANDOM_STATE)  # 'frac=1' mischt den gesamten DataFrame
df_balanced_reviews_sentences.reset_index(drop=True, inplace=True)

### Check If Sentences are Unique

In [12]:
df_balanced_reviews_sentences.groupby(["review_id", "sentence_idx"]).size().reset_index(name="count")

,review_id,sentence_idx,count
0,845636774,0,1
1,845636774,1,1
2,845636774,2,1
3,845636774,3,1
4,845705523,0,1
...,...,...,...
3495,916494891,0,1
3496,916494891,3,1
3497,916494891,4,1
3498,916494891,7,1


### Store as .csv 

In [13]:
df_reviews_sentences.to_csv("reviews_dataset/reviews_sentences.csv")

In [14]:
df_balanced_reviews_sentences.to_csv("reviews_dataset/balanced_reviews_sentences.csv")

In [15]:
df_balanced_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
0,1906,875871390,8009090,16,Hausgemachte frische Pasta mit viel Auswahl un...,2023-01-21,Culture08862634821,"Dresden, Deutschland",Im Menü gibt es jeweils noch ein Getränk.,4.0,dresden,Mamma Mia,de,Ein solides italienisches Restaurant in der Dr...,15,c04b01b6-3792-40f2-85a7-0a1f8e10ca53,de
1,2445,910308867,1342313,1,In jeder Hinsicht toll,2023-08-14,sandraburmester,"Lingen, Deutschland",Wir hatten einen einzigartig schönen Abend im ...,5.0,bielefeld,Jivino Enoteca,de,Wir hatten einen einzigartig schönen Abend im ...,0,4853b49d-cde6-4419-b9c2-f820792477fa,de
2,2398,905245137,7381559,0,Leider zukünftig nicht mehr...,2023-07-24,Rolfitruck,"Velbert, Deutschland","Allerdings stimmt, aus unserer Sicht, das Prei...",2.0,wuppertal,Amalfi Il Ristorante,de,Wir waren nun innerhalb der letzten 3 Wochen z...,3,6578de2e-9a0a-4b4e-875a-af84b56ddf6b,de
3,723,859195793,1190751,12,So lecker wie immer,2022-09-09,annaliebrern,NaN,"Immer vorbestellt Menüs, für jeden was dabei.",5.0,frankfurt,Trattoria i Siciliani,de,Sie kochen für unsere Geburtstagsfeiern seit J...,1,e7979473-4e7b-4ed1-b57e-178a172974f5,de
4,563,850412704,718270,1,Lecker Kölsch,2022-07-28,kaiserdirk,"Leverkusen, Deutschland",Wir mögen Peters Kölsch.2.,4.0,köln,Peters Brauhaus,de,"Hier kehren wir immer wieder gerne ein, aus dr...",1,c67870b8-b5e3-40f0-8a60-1de08d61d0fc,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,2151,902708504,1118853,0,"Herzhaft, bayrisch, lecker!",2023-07-13,DayTrip36564843976,Deutschland,Auch dieses Restaurant kann ich nur weiterempf...,4.0,hannover,Bavarium,de,Heute war ich mit meiner Familie hier und wir ...,5,3bb03dba-c743-4a97-bb46-76f6ed29a772,de
3496,3592,893275773,5207419,0,So gefällt Geburtstag feiern außer Haus,2023-06-03,Iris S,NaN,"Zum Servicepersonal, der Küche und dem „Chef i...",4.0,mönchengladbach,Brauerei zum Stefanus,de,Dem Anlass angemessen (Geburtstag) waren wir P...,6,6ecb5ec5-daf7-4bea-be68-0d41504bf4bf,de
3497,550,888907029,718270,0,Lecker Bier und tolle Atmosphäre,2023-05-04,Axel S,"Braunschweig, Deutschland",Wir waren hier zum Abendessen.,4.0,köln,Peters Brauhaus,de,Wir waren hier zum Abendessen. Atmosphäre ist ...,0,d7c13500-9a94-49c2-9596-9498be50faec,de
3498,3175,889498235,7322855,4,Ein Reines Gedicht,2023-05-08,Tim R,NaN,Hauptspeise:Wir haben uns beide für das Prime ...,5.0,augsburg,SteakManufaktur,de,Zur Ambiente:Wir würden herzlich empfangen vom...,9,6910956d-2a8f-48aa-8209-71d7ba19c79e,de
